In the previous Jupyter Notebook, I used TPOT, an autoML package, to discover the lowest MSE model for predicting RMSD of proteins based on their physiochemical properties. In this Notebook I use the selected model and implement it as a Dash application. For Predicting RMSD based on protein characteristics in a Dash App, I first import necessary packages

In [1]:
pip install keras

In [2]:
pip install jupyter-dash -q #

     |████████████████████████████████| 9.6 MB 13.4 MB/s 
     |████████████████████████████████| 357 kB 46.8 MB/s 


In [3]:
## Imports
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense
import joblib
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc 
import dash_html_components as html
from dash.dependencies import Input, Output

from keras.models import load_model
import joblib

import numpy as np
import pandas as pd

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`


In [ ]:
from google.colab import files
uploaded = files.upload()
#caspdf = pd.read_csv('CASP (1).csv')

Saving CASP.csv to CASP (2).csv


Next, I load the dataset into a pandas dataframe 'caspdf'

In [5]:
caspdf = pd.read_csv('CASP.csv')

Below I create a Extra Trees Regressor model that was selected in the last notebook by comparing multiple models with TPOT. This model had the best performance with the parameters used below.

In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

X = caspdf.drop(['RMSD'], axis=1)
y = caspdf['RMSD']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

# Average CV score on the training set was: -12.126678354624435
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    ExtraTreesRegressor(bootstrap=False, max_features=0.9000000000000001, min_samples_leaf=1, min_samples_split=5, n_estimators=100)
)

exported_pipeline.fit(X_train, y_train)
y_pred_extra_trees = exported_pipeline.predict(X_test)


After fitting the model I evaluate the model on R squared, MSE, and RMSE.

In [7]:
#Calculating R squard
from sklearn.metrics import r2_score
r2 = r2_score(y_true=y_test, y_pred=y_pred_extra_trees)
print("R-squared for Extra Trees Regressor model 1 is: {:,.2F}".format(r2))

R-squared for Extra Trees Regressor model 1 is: 0.71


In [8]:
from sklearn.metrics import mean_squared_error
mse_extra_trees = mean_squared_error(y_true=y_test, y_pred=y_pred_extra_trees)
print('MSE for Extra Trees Regressor model is {:0.2f}'.format(mse_extra_trees))

MSE for Extra Trees Regressor model is 11.06


In [9]:
from sklearn.metrics import mean_squared_error
rmse_extra_trees = (mean_squared_error(y_true=y_test, y_pred=y_pred_extra_trees))**0.5
print('RMSE for Extra Trees Regressor model is {:0.2f}'.format(rmse_extra_trees))

RMSE for Extra Trees Regressor model is 3.33


In [10]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

     |████████████████████████████████| 102 kB 31.6 MB/s 
     |████████████████████████████████| 47.8 MB 2.2 MB/s 
albumentations==1.1.0 is successfully installed


In [11]:
extra_trees_regressor = ExtraTreesRegressor(bootstrap=False, max_features=0.9000000000000001, min_samples_leaf=1, min_samples_split=5, n_estimators=100)
extra_trees_regressor.fit(X_train, y_train)
y_pred_extra_trees_2 = extra_trees_regressor.predict(X_test)

In [12]:
#Calculating R squard
from sklearn.metrics import r2_score
r2 = r2_score(y_true=y_test, y_pred=y_pred_extra_trees_2)
print("R-squared for Extra Trees Regressor model 1 is: {:,.2F}".format(r2))

R-squared for Extra Trees Regressor model 1 is: 0.70


In [ ]:
import albumentations as A

In [14]:
joblib.dump(extra_trees_regressor, "extra-trees-regressor")

['extra-trees-regressor']

In [15]:
pip install jupyter-dash -q #

In [16]:
import dash
from jupyter_dash import JupyterDash

import dash_core_components as dcc 
import dash_html_components as html
from dash.dependencies import Input, Output

from keras.models import load_model
import joblib

import numpy as np
import pandas as pd

In [17]:
app = JupyterDash(__name__)
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})


In [18]:
import pickle


In [19]:
filename = 'extra-trees-regressor'
pickle.dump(extra_trees_regressor, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
model = joblib.load('extra-trees-regressor')
#model = ('extra-trees-regressor')

Below I create the figures for the Dash application

In [20]:
fig1 = px.scatter(caspdf, x="RMSD", y="F3", title= "RMSD vs. F3, Correlation: +0.374", trendline="ols", trendline_color_override='#4FFC00')
fig1.update_layout(clickmode='event+select')
fig1.update_traces(marker_size=15)

fig2 = px.scatter(caspdf, x="RMSD", y="F4", title="RMSD vs F4, Correlation: -0.170", trendline="ols", trendline_color_override='#4FFC00')
fig2.update_layout(clickmode='event+select')
fig2.update_traces(marker_size=15)

fig3 = px.scatter(caspdf, x="RMSD", y="F2", title="RMSD vs F2, Correlation: +0.157", trendline="ols", trendline_color_override='#4FFC00')
fig3.update_layout(clickmode='event+select')
fig3.update_traces(marker_size=15)

fig4 = px.scatter(caspdf, x="F3", y="F2", title= "F3 vs. F2, Correlation: +0.503", trendline="ols", trendline_color_override='#4FFC00')
fig4.update_layout(clickmode='event+select')
fig4.update_traces(marker_size=15)

#app.layout= html.Div([
#    dcc.Graph(
#        id='F1 and RMSD',
#        figure=fig
#    )
#])

#app.layout = html.Div(children = [

#        dcc.Graph(id='my-histogram', figure=figure)
#])

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [21]:
import dash_table

# Correlation DataFrame
corr_df = pd.DataFrame(caspdf.corr())
corr_df = corr_df.round(decimals=4)

Correlation_df = html.Div([
       dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in corr_df.columns],
    data=corr_df.to_dict('records'),
)                    
])


# 10 item sample DataFrame
sample_df = pd.DataFrame(caspdf.sample(n=10, random_state=123))
sample_df = sample_df.round(4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning:


The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`



In [22]:
def generate_table(dataframe, max_rows=10):
  return html.Table([
      html.Thead(
          html.Tr([html.Th(col) for col in dataframe.columns])
      ),
      html.Tbody([
            html.Tr([ 
                     html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
      ])
  ])

Correlation_df = html.Div([
    html.Label("Correlation Values:"),                       
    dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in corr_df.columns],
    data=corr_df.to_dict('records'),
)                    
])

 ## Div for carat
RMSD_F3_graph = html.Div([
    dcc.Graph(
        id='RMSD and F3',
        figure=fig1
    )
])
RMSD_F4_graph = html.Div([
    dcc.Graph(
        id='RMSD and F4',
        figure=fig2
    )
])
RMSD_F2_graph = html.Div([
    dcc.Graph(
        id='RMSD and F2',
        figure=fig3
    )
])
F3_F2_graph = html.Div([
    dcc.Graph(
        id='F3 and F2',
        figure=fig4
    )
])


input_F1 = dcc.Input(
    id='F1',
    type='number',
    value=1)

div_F1 = html.Div(
    children=[html.H3('F1:'), input_F1],
    className="four columns"
    )

input_F2 = dcc.Input(
    id='F2',
    type='number',
    value=1)

div_F2 = html.Div(
    children=[html.H3('F2:'), input_F2],
    className="four columns"
    )

input_F3 = dcc.Input(
    id='F3',
    type='number',
    value=1)

div_F3 = html.Div(
    children=[html.H3('F3:'), input_F3],
    className="four columns"
    )

input_F4 = dcc.Input(
    id='F4',
    type='number',
    value=1)

div_F4 = html.Div(
    children=[html.H3('F4:'), input_F4],
    className="four columns"
    )

input_F5 = dcc.Input(
    id='F5',
    type='number',
    value=1)

div_F5 = html.Div(
    children=[html.H3('F5:'), input_F5],
    className="four columns"
    )

input_F6 = dcc.Input(
    id='F6',
    type='number',
    value=1)

div_F6 = html.Div(
    children=[html.H3('F6:'), input_F6],
    className="four columns"
    )

input_F7 = dcc.Input(
    id='F7',
    type='number',
    value=1)

div_F7 = html.Div(
    children=[html.H3('F7:'), input_F7],
    className="four columns"
    )

input_F8 = dcc.Input(
    id='F8',
    type='number',
    value=1)

div_F8 = html.Div(
    children=[html.H3('F8:'), input_F8],
    className="four columns"
    )

input_F9 = dcc.Input(
    id='F9',
    type='number',
    value=1)

div_F9 = html.Div(
    children=[html.H3('F9:'), input_F9],
    className="four columns"
    )



In [23]:
## Div for numerical characteristics
div_numerical = html.Div(
    children = [div_F1, div_F2, div_F3, div_F4, div_F5, div_F6, div_F7, div_F8, div_F9],
    className="row"
    )
div_graphical = html.Div(
    children = [Correlation_df, RMSD_F3_graph, RMSD_F4_graph, RMSD_F2_graph, F3_F2_graph],
    className="row"
    )


Below I create a function for getting new predictions.

In [24]:
def get_prediction(F1, F2, F3, F4, F5, F6, F7, F8, F9):
    '''takes the inputs from the user and produces the RMSD prediction'''
    
    cols = ['F1', 'F2','F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9']

    #cut_dict = {x: 'cut_' + x for x in cut_values[1:]}
    #color_dict = {x: 'color_' + x for x in color_values[1:]}
    #clarity_dict = {x: 'clarity_' + x for x in clarity_values[1:]}
    
    ## produce a dataframe with a single row of zeros
    df = pd.DataFrame(data = np.zeros((1,len(cols))), columns = cols)
    
    ## get the numeric characteristics
    df.loc[0,'F1'] = F1
    df.loc[0,'F2'] = F2
    df.loc[0,'F3'] = F3 
    df.loc[0,'F4'] = F4
    df.loc[0,'F5'] = F5
    df.loc[0,'F6'] = F6
    df.loc[0,'F7'] = F7
    df.loc[0,'F8'] = F8
    df.loc[0,'F9'] = F9

    ## transform dimensions into a single dim_index using PCA
    #dims_df = pd.DataFrame(data=[[x, y, z]], columns=['x','y','z'])
    #df.loc[0,'dim_index'] = pca.transform(dims_df).flatten()[0]
    
    ## Use the one-hot encoding for the categorical features
    #if cut!='Fair':
    #    df.loc[0, cut_dict[cut]] = 1
    
    #if color!='D':
    #    df.loc[0, color_dict[color]] = 1
    
    #if clarity != 'I1':
    #    df.loc[0, clarity_dict[clarity]] = 1
    
    ## Scale the numerical features using the trained scaler
    #numerical_features = ['carat', 'depth', 'table', 'dim_index']
    #df.loc[:,numerical_features] = scaler.transform(df.loc[:,numerical_features])
    
    ## Get the predictions using our trained neural network
    prediction = loaded_model.predict(df.values).flatten()[0]
    
    ## Transform the log-prices to prices
    #prediction = np.exp(prediction)
    return int(prediction)

In [25]:
markdown_text = '''
### RMSD-Size of the residue. \

F1 - Total surface area. \

F2 - Non polar exposed area. \

F3 - Fractional area of exposed non polar residue. \

F4 - Fractional area of exposed non polar part of residue. \

F5 - Molecular mass weighted exposed area. \

F6 - Average deviation from standard exposed area of residue. \

F7 - Euclidian distance. \

RMSD is a measure of how well a predicted protein's structure fits to an experimental structure, with a value of zero being a perfect fit. F1-F7 correspond to other features about the protein's structure. The goal here is to use the other numerical features to predict the target value RMSD. 

TPOT Analysis was done to find a strong model for modelling. Extra Trees Regressor Model was used. It had the following test dataset (10% of total data) metrics:

R-squared for Extra Trees Regressor model 1 is: 0.71

MSE for Extra Trees Regressor model is 11.03

RMSE for Extra Trees Regressor model is 3.32

Below sample values can be input to get a prediction for RMSD. There are plots showing the relationship between RMSD and the three most correlated features, as well as a plot of F3 vs F2, as F2 is the strongest correlator to RMSD.

'''


In [26]:
colors = {
    'background1': '#4C92FA',
    'text': '#7FDBFF',
    'background2': '#74ABFF'
}
#app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
#    html.H1(    return int(prediction)

## App layout

#html.Div(style={'backgroundColor': colors['background']}, children=[

app.layout = html.Div(children=[


       
        html.H1('Predict RMSD from Protein Physiochemical Properties'),
        
        html.Div([dcc.Markdown(children=markdown_text)]),

        html.H2('Enter the protein characteristics to get the predicted RMSD'),
        
        html.H1(id='output',
                style={'backgroundColor': colors['background2'], 'margin-top': '50px', 'text-align': 'center'}),
        
        html.H4(children='Sample Values'), generate_table(sample_df),

        html.Div(
                children=[div_numerical, div_graphical]
                )           
#        html.Div(
#                children=[RMSD_F1_graph]
#                ),           
        ])

In [27]:
predictors = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9']
@app.callback(
        Output('output', 'children'),
        [Input(x, 'value') for x in predictors])
def show_prediction(F1, F2, F3, F4, F5, F6, F7, F8, F9): 
    pred = get_prediction(F1, F2, F3, F4, F5, F6, F7, F8, F9)
    return str("Predicted RMSD: {:,}".format(pred))


Below I initialize the Dash app.

In [28]:
if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


/usr/local/lib/python3.7/dist-packages/dash/resources.py:72: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



<IPython.core.display.Javascript object>